# Order Imbalance Strategy in Chinese Future Markets

# Introduction
This project is to replicate the paper from D.Shen(2015), who analysed the factor 'Order imbalance' to have a significant impact on the change in CSI300 Futures tick data backed in 2014. Here are his findings:

1. Developed a trading strategy by fitting the order imbalance factor model to forecast 10 seconds mid price change.
2. Model shows high profitability when trading signal is greater than 0.2 ticks.
3. Profits made by the strategy are strongly correlated with the total trading volume in the market

Additional findings:
1. Improved trading signal by extending model to 2 more factors : Order imbalance ratio and mid price basis mean reversion, and divided variables by bid-ask spread. 
2. Large spread indicates low price change
3. Trading parameters optimization : Forecast window for average change and trading threshold were closer to 5 and 0.15 respectively

Results from D.Shen(2015):
1. Average Correlation coefficeint of OI and instantaneous mid-price change : 0.3935
2. Fitting a linear regression model gives average daily R-squared : 0.155
3. Correlation between 10s interval between price change and VOI R-squared: 0.6537
4. Variable samples proved to be stationary by ADF/KPSS test

# Step 1 : Identifying Objectives
The first step of this project is to define what we are looking for. In my own understanding, the whole research can guide us to potential trading strategy and we are about to find out whether the key factor fits well with the data. D.Shen(2015) has proven a significant amount of profit backing up by the linear model. That means the signal(alpha) has a robust performance in the CSI300 Future tick data. The next step is to perform data cleaning. It is crucially important because uncleaned data may lead to irregular results.

# Step 2 : Checking & tidying data
Facts about the SHFE tick data set:
1. Tick data from 00:00:00.000 to 23:59:59.500 GMT+8
2. Interval per each entry : 5ms difference(Snapshot data)
3. 16 Future contracts with 12 expiries & Multiple Option contracts(Not included in this research). e.g. RB1912 to RB2012
4. LEVEL 2 quote.
5. Volume and Turnover resets at 21:00:00.000 for all contracts, which means, volume and turnover counts from previous day's 21:00:00.000 for contracts that trades during night session.

Let us import the dataset and see what its looks like:

In [1]:
import numpy as np
import os
import pandas as pd
import statsmodels.api as sm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
#read file
df = pd.read_csv('ShfeMulti_20191031.csv', index_col=None, header=None)
df.columns = ['Timestamp','Contract', 'time','last_price','volume','turnover','open_interest', #rename column
              'BP1','BV1','AP1','AV1',
              'BP2','BV2','AP2','AV2',
              'BP3','BV3','AP3','AV3',
              'BP4','BV4','AP4','AV4',
              'BP5','BV5','AP5','AV5'] 
df.head(10)

,Timestamp,Contract,time,last_price,volume,turnover,open_interest,BP1,BV1,AP1,AV1,BP2,BV2,AP2,AV2,BP3,BV3,AP3,AV3,BP4,BV4,AP4,AV4,BP5,BV5,AP5,AV5
0,3.240447e+09,ag1911,20:59:00.500,4321.0,0.0,0.0,324.0,4282.0,2.0,4396.0,1.0,4246.0,1.0,4424.0,2.0,4216.0,2.0,0.0,0.0,4135.0,2.0,0.0,0.0,4078.0,1.0,0.0,0.0
1,3.240447e+09,ag1912,20:59:00.500,4359.0,4224.0,276186240.0,815008.0,4358.0,41.0,4359.0,232.0,4357.0,44.0,4360.0,280.0,4356.0,288.0,4361.0,387.0,4355.0,181.0,4362.0,212.0,4354.0,43.0,4363.0,220.0
2,3.240447e+09,ag2001,20:59:00.500,4377.0,0.0,0.0,2386.0,4360.0,1.0,4397.0,10.0,4358.0,6.0,4398.0,6.0,4357.0,10.0,4399.0,1.0,4334.0,1.0,4461.0,1.0,4319.0,1.0,4470.0,1.0
3,3.240447e+09,ag2002,20:59:00.500,4390.0,274.0,18042900.0,339136.0,4388.0,6.0,4391.0,6.0,4387.0,1.0,4392.0,7.0,4386.0,1.0,4393.0,7.0,4385.0,8.0,4394.0,13.0,4384.0,15.0,4395.0,88.0
4,3.240447e+09,ag2003,20:59:00.500,4405.0,0.0,0.0,490.0,4358.0,1.0,4500.0,1.0,4343.0,1.0,4585.0,1.0,4188.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3.240447e+09,ag2004,20:59:00.500,4418.0,30.0,1988100.0,177686.0,4413.0,1.0,4418.0,11.0,4402.0,15.0,4422.0,5.0,4400.0,7.0,4424.0,3.0,4398.0,5.0,4429.0,15.0,4394.0,8.0,4434.0,8.0
6,3.240447e+09,ag2005,20:59:00.500,4431.0,0.0,0.0,232.0,4382.0,1.0,4524.0,1.0,4188.0,1.0,4585.0,1.0,4173.0,1.0,4649.0,1.0,4169.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN
7,3.240447e+09,ag2006,20:59:00.500,4413.0,430.0,28463850.0,163504.0,4413.0,10.0,4440.0,1.0,4411.0,5.0,4443.0,3.0,4410.0,1.0,4444.0,2.0,4408.0,3.0,4445.0,20.0,4405.0,1.0,4446.0,24.0
8,3.240447e+09,ag2007,20:59:00.500,4441.0,0.0,0.0,190.0,4408.0,1.0,4550.0,1.0,4377.0,1.0,4567.0,1.0,4228.0,1.0,4638.0,1.0,4197.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN
9,3.240447e+09,ag2008,20:59:00.500,4455.0,0.0,0.0,756.0,4440.0,1.0,4493.0,1.0,4420.0,1.0,4513.0,1.0,4400.0,1.0,4533.0,1.0,4380.0,1.0,4553.0,1.0,4360.0,1.0,4573.0,1.0


There are some data cleaning procedures before we continue our analysis. 
1. The time is not in datetime format so we need to convert it to that. 
2. Filtering out option contracts because we are not comparing them in this project
3. Obtain only for Active contracts for the highest volume
4. NaN values need to be removed

In [2]:
import datetime as dt
pd.set_option('display.float_format', '{:.2f}'.format) #Set display decimals


#Convert datetime object as Index
df = df.sort_values(['time'], ascending=True) #Sort time
df.time  = pd.to_datetime(df.time) #Convert time to datetime object
df.time  = pd.to_datetime(df.time).dt.strftime('%H:%M:%S.%f')##Remove Y/M/D , I only want H/M/S 

#obtain time from 09:00 - 15:00, Not including night session
df = df[(df.time >= '08:59:00.000') & (df.time <='15:00:00.500')]
df = df.set_index('time')#Set time as index

#Obtaining Active contract names 
#df = df[['Contract','volume','turnover', 'BP1','BV1','AP1','AV1']]
#filter out option contracts, more than 6 characters are options contracts
df = df[df['Contract'].str.len() <= 6]
#Filter in Active Contract, all contracts doesn't work...
df['Contract1'] = df.Contract.str.slice(start=0,stop =2)
df['Contract2'] = df.Contract.str.slice(start=2)
temp = df.groupby('Contract').max().reset_index()
temp= temp[['volume','Contract1','Contract']]
temp = temp[temp.volume == temp.groupby('Contract1')['volume'].transform('max')]
main = temp.Contract.to_numpy()
df = df[df['Contract'].isin(main)] 
df = df.sort_values(['time','Contract'], ascending=True)

#Remove NaNs 
df.dropna(inplace = True)

df.sort_values(['Contract1'], ascending=True)

df.head(25)

,Timestamp,Contract,last_price,volume,turnover,open_interest,BP1,BV1,AP1,AV1,BP2,BV2,AP2,AV2,BP3,BV3,AP3,AV3,BP4,BV4,AP4,AV4,BP5,BV5,AP5,AV5,Contract1,Contract2
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,
08:59:00.500000,3283647182.29,wr2001,3660.00,0.00,0.00,30.00,3636.00,1.00,3690.00,1.00,3620.00,1.00,3720.00,1.00,3606.00,1.00,3760.00,1.00,3566.00,1.00,3786.00,1.00,3506.00,1.00,3880.00,2.00,wr,2001
09:00:00.500000,3283707180.75,ag1912,4352.00,663578.00,43350185370.00,811918.00,4351.00,110.00,4352.00,24.00,4350.00,1032.00,4353.00,5.00,4349.00,43.00,4354.00,62.00,4348.00,77.00,4355.00,18.00,4347.00,32.00,4356.00,451.00,ag,1912
09:00:00.500000,3283707180.77,al1912,13815.00,22590.00,1561775000.00,230674.00,13810.00,52.00,13815.00,13.00,13805.00,338.00,13820.00,68.00,13800.00,81.00,13825.00,29.00,13795.00,108.00,13830.00,59.00,13790.00,67.00,13835.00,186.00,al,1912
09:00:00.500000,3283707180.78,au1912,341.20,181304.00,61837871700.00,347730.00,341.05,7.00,341.20,15.00,341.00,6.00,341.25,53.00,340.95,4.00,341.30,5.00,340.90,15.00,341.35,10.00,340.85,102.00,341.40,8.00,au,1912
09:00:00.500000,3283707180.80,bu1912,2948.00,335662.00,9884005640.00,430862.00,2944.00,51.00,2948.00,2.00,2942.00,201.00,2950.00,47.00,2940.00,237.00,2952.00,158.00,2938.00,214.00,2954.00,149.00,2936.00,251.00,2956.00,254.00,bu,1912
09:00:00.500000,3283707180.82,cu1912,47420.00,40374.00,9583929000.00,228344.00,47410.00,2.00,47420.00,9.00,47400.00,59.00,47430.00,1.00,47390.00,12.00,47440.00,66.00,47380.00,56.00,47460.00,1.00,47370.00,17.00,47470.00,3.00,cu,1912
09:00:00.500000,3283707180.85,fu2001,2093.00,422360.00,8881604260.00,760108.00,2093.00,19.00,2094.00,110.00,2092.00,7.00,2095.00,16.00,2091.00,164.00,2096.00,3.00,2090.00,363.00,2097.00,30.00,2089.00,492.00,2098.00,15.00,fu,2001
09:00:00.500000,3283707180.86,hc2001,3378.00,130152.00,4391675200.00,635200.00,3376.00,76.00,3378.00,10.00,3375.00,10.00,3379.00,503.00,3374.00,39.00,3380.00,176.00,3373.00,83.00,3381.00,19.00,3372.00,79.00,3382.00,7.00,hc,2001
09:00:00.500000,3283707180.87,ni1912,133900.00,271812.00,36422559200.00,238168.00,133880.00,6.00,133950.00,53.00,133840.00,17.00,133970.00,8.00,133830.00,61.00,133980.00,1.00,133820.00,9.00,134000.00,3.00,133810.00,13.00,134010.00,1.00,ni,1912


# Step 2 : Factor Analysis
The key signal in this project is 'Volume Order Imbalance'(VOI) and Midprice. Let's compute it.

In [3]:
#Compute VOI, the signal
def VOI(df):
    delta_VB = [0];delta_VA = [0]
    t=0;u=0
    for t in range(1,len(df.index)):
        if df.BP1[t] < df.BP1[t-1]:
            delta_VB.append(0)
        elif df.BP1[t] == df.BP1[t-1]:
            delta_VB.append(df.BV1[t] - df.BV1[t-1])
        elif df.BP1[t] > df.BP1[t-1]:
            delta_VB.append(df.BV1[t])
    for u in range(1,len(df.index)):
        if df.AP1[t] < df.AP1[t-1]:
            delta_VA.append(df.AV1[t])
        elif df.AP1[t] == df.AP1[t-1]:
            delta_VA.append(df.AV1[t] - df.AV1[t-1])
        elif df.AP1[t] > df.AP1[t-1]:
            delta_VA.append(0)
    VOI = np.subtract(delta_VB,delta_VA)
    return VOI

#Mid price , or we can fully utilitize the order book, VWAP(next step)
def midprice(df):
    midprice = (df.BP1 +df.AP1)/2
    return midprice

#Bid Ask Spread
def BAS(df):
    return (df.AP1 - df.BP1)

### Statistical Analysis 
For time series data analysis, a few things need to be checked before we run the linear model. 

1. Normalization of midprice change
2. Autocorrelation a.k.a. Serial Correlation : Check similarity within time series
3. Stationarity (ADF test), Series can steadily accumulate positive returns. Reject Null Hypothesis if the series is stationary. 

Before we check those, we need to normalize the midprice change by their tick size since we are comparing data within contracts.

In [4]:
rb = df[df.Contract1== 'rb']
#Obtaining ticksize
def ticksize(df):
    x=df['BP1'].diff()
    x = x.replace(0, np.nan)
    x.dropna(inplace=True)
    return abs(x.value_counts().idxmax())

temp = df.groupby('Contract').apply(midprice).diff()/df.groupby('Contract').apply(ticksize)
df.groupby('Contract').apply(ticksize)

Contract
ag1912    1.00
al1912    5.00
au1912    0.05
bu1912    2.00
cu1912   10.00
fu2001    1.00
hc2001    1.00
ni1912   10.00
pb1912    5.00
rb2001    1.00
ru2001    5.00
sn2001   10.00
sp2001    2.00
ss2002    5.00
wr2001    1.00
zn1912    5.00
dtype: float64

Now have our change in midprice normalized against ticksize. We can conduct our analysis. The first one is **Correlation**. 

In [5]:
pd.set_option('display.float_format', '{:.4f}'.format)
def cor(df):
    cor = np.corrcoef(np.diff(midprice(df))/ticksize(df),VOI(df)[1:])[1,0]
    return cor

correlation_table = pd.DataFrame(df.groupby('Contract').apply(cor))
print(correlation_table)

              0
Contract       
ag1912   0.1921
al1912   0.1787
au1912   0.1960
bu1912   0.1670
cu1912   0.1262
fu2001   0.3462
hc2001   0.1892
ni1912   0.2678
pb1912   0.2379
rb2001   0.1627
ru2001   0.2208
sn2001   0.2340
sp2001   0.2228
ss2002   0.1233
wr2001   0.1190
zn1912   0.1899


### Autocorrelation 


In [6]:
#Autocorrelation coefficient for dM with lag from 0 to 5
def dM_autocor(df):
    temp = midprice(df).diff()/ticksize(df)
    autocor = [round(temp.autocorr(lag = i),2) for i in range(0,6)]
    return autocor

autocorrr = df.groupby('Contract').apply(dM_autocor)
autocorrr

Contract
ag1912    [1.0, -0.11, -0.05, -0.03, -0.01, -0.02]
al1912     [1.0, -0.07, -0.1, -0.08, -0.01, -0.03]
au1912     [1.0, -0.19, -0.06, -0.04, -0.03, -0.0]
bu1912    [1.0, -0.08, -0.08, -0.02, -0.01, -0.02]
cu1912    [1.0, -0.18, -0.04, -0.01, -0.03, -0.03]
fu2001    [1.0, -0.11, -0.05, -0.02, -0.04, -0.01]
hc2001    [1.0, -0.23, -0.03, -0.01, -0.01, -0.01]
ni1912         [1.0, -0.2, 0.02, 0.03, 0.01, -0.0]
pb1912    [1.0, -0.16, -0.02, -0.05, -0.02, -0.02]
rb2001    [1.0, -0.14, -0.04, -0.01, -0.02, -0.02]
ru2001    [1.0, -0.05, -0.06, -0.04, -0.02, -0.01]
sn2001      [1.0, -0.18, 0.05, -0.01, 0.02, -0.01]
sp2001      [1.0, -0.16, -0.05, 0.02, -0.05, 0.02]
ss2002     [1.0, -0.26, -0.05, 0.06, -0.01, -0.03]
wr2001      [1.0, -0.01, -0.09, 0.01, -0.17, -0.0]
zn1912     [1.0, -0.18, -0.06, -0.0, -0.01, -0.02]
dtype: object

In [7]:
#Autocorrelation coefficient for VOI with lag from 0 to 5
def VOI_autocor(df):
    temp = VOI(df).diff()/ticksize(df)
    autocor = [round(temp.autocorr(lag = i),2) for i in range(0,6)]
    return autocor

autocorrr = df.groupby('Contract').apply(dM_autocor)
autocorrr

Contract
ag1912    [1.0, -0.11, -0.05, -0.03, -0.01, -0.02]
al1912     [1.0, -0.07, -0.1, -0.08, -0.01, -0.03]
au1912     [1.0, -0.19, -0.06, -0.04, -0.03, -0.0]
bu1912    [1.0, -0.08, -0.08, -0.02, -0.01, -0.02]
cu1912    [1.0, -0.18, -0.04, -0.01, -0.03, -0.03]
fu2001    [1.0, -0.11, -0.05, -0.02, -0.04, -0.01]
hc2001    [1.0, -0.23, -0.03, -0.01, -0.01, -0.01]
ni1912         [1.0, -0.2, 0.02, 0.03, 0.01, -0.0]
pb1912    [1.0, -0.16, -0.02, -0.05, -0.02, -0.02]
rb2001    [1.0, -0.14, -0.04, -0.01, -0.02, -0.02]
ru2001    [1.0, -0.05, -0.06, -0.04, -0.02, -0.01]
sn2001      [1.0, -0.18, 0.05, -0.01, 0.02, -0.01]
sp2001      [1.0, -0.16, -0.05, 0.02, -0.05, 0.02]
ss2002     [1.0, -0.26, -0.05, 0.06, -0.01, -0.03]
wr2001      [1.0, -0.01, -0.09, 0.01, -0.17, -0.0]
zn1912     [1.0, -0.18, -0.06, -0.0, -0.01, -0.02]
dtype: object

We can see autocorrelation for both exogenous and endogenous variable all vanish to zero as lag increases. Hence it shows that the time series is persistent.

### Stationarity
The first values gives the ADF test statistic, the second value returns the P-value. If P-value is 0.000 , that means reject Null hypothesis at 99%. Thus the time series is stationary.

In [8]:
from statsmodels.tsa.stattools import adfuller
#For dM
def dM_adf_test(df):
    x = midprice(df).diff()[1:]
    result = adfuller(x)
    l = [round(result[0],4), round(result[1],4)]
    return l

df.groupby('Contract').apply(dM_adf_test)

Contract
ag1912    [-44.1018, 0.0]
al1912    [-24.8968, 0.0]
au1912    [-26.8083, 0.0]
bu1912    [-36.6114, 0.0]
cu1912    [-37.5743, 0.0]
fu2001    [-34.5356, 0.0]
hc2001    [-43.0506, 0.0]
ni1912    [-51.5021, 0.0]
pb1912    [-39.2873, 0.0]
rb2001    [-48.0519, 0.0]
ru2001    [-45.0737, 0.0]
sn2001    [-64.3324, 0.0]
sp2001    [-21.3623, 0.0]
ss2002    [-48.2938, 0.0]
wr2001     [-9.5911, 0.0]
zn1912    [-32.0288, 0.0]
dtype: object

In [9]:
#For VOI
def VOI_adf_test(df):
    x = VOI(df)
    result = adfuller(x)
    l = [round(result[0],4), round(result[1],4)]
    return l
df.groupby('Contract').apply(VOI_adf_test)

Contract
ag1912     [-83.678, 0.0]
al1912    [-26.6563, 0.0]
au1912    [-65.1832, 0.0]
bu1912    [-20.2048, 0.0]
cu1912    [-48.9369, 0.0]
fu2001    [-98.1595, 0.0]
hc2001     [-24.282, 0.0]
ni1912    [-35.6773, 0.0]
pb1912    [-59.6626, 0.0]
rb2001    [-20.5119, 0.0]
ru2001    [-34.2519, 0.0]
sn2001    [-13.8988, 0.0]
sp2001    [-31.3668, 0.0]
ss2002    [-53.8318, 0.0]
wr2001     [-5.7293, 0.0]
zn1912    [-36.8564, 0.0]
dtype: object

# Step 4 : Linear Modelling

In the next step I ran the linear OLS model to check how well does the model fit the snapshot data. We are separating the approach to a few stages : 
1. Benchmark Model (Random Walk)
$$ \Delta M_t = \alpha + \epsilon _t $$
2. Fit VOI as exogenous variable 
$$ \Delta M_t = \alpha + \beta OI_t + \epsilon  _t $$
3. Multifactor Model
$$ \Delta M_t = \alpha + \beta_{OI} \frac{OI_t}{S_t} + \beta_{OIR} \frac{OIR_t}{S_t} + \beta_{MPR} \frac{MPR_t}{S_t} + \epsilon  _t $$


### Random Walk Model (Benchmark)
In the following results, the following sequence of number represents as:

$$[ \alpha, t(\alpha)  , R^2 , Adj  R^2 ]$$

The MSE (Mean Squared Error) can specified as:
$$ MSE = \frac{1}{n} \sum_{k=1}^{n} (y - y_{pred})^2 $$

In [127]:
def ols(df):
    y = midprice(df).diff()[1:]/ticksize(df)
    x = np.array([1]*(len(df)-1))
    res = sm.OLS(y,x).fit()
    pred = res.predict(x)
    liss = pd.Series([round(res.params[0],4),round(res.tvalues[0],4), round(res.rsquared,6), round(res.rsquared_adj,6), np.mean((y - pred)**2)])
    return liss

model = df.groupby('Contract').apply(ols)
model.columns = ['Alpha', 't(alpha)','R^2','Adj R^2','MSE']
model

,Alpha,t(alpha),R^2,Adj R^2,MSE
Contract,,,,,
ag1912,0.0006,0.4269,-0.0000,-0.0000,0.0495
al1912,0.0005,0.3234,0.0000,0.0000,0.0179
au1912,0.0000,0.0156,0.0000,0.0000,0.0594
bu1912,0.0006,0.6518,0.0000,0.0000,0.0170
cu1912,-0.0006,-0.3616,-0.0000,-0.0000,0.0395
fu2001,0.0006,0.5687,0.0000,0.0000,0.0247
hc2001,-0.0012,-0.5848,0.0000,0.0000,0.0860
ni1912,-0.0009,-0.1500,0.0000,0.0000,0.8353
pb1912,-0.0017,-0.5228,-0.0000,-0.0000,0.0859


### VOI model
The following results are as follows:

$$[ \alpha, t(\alpha)  , \beta , t(\beta),  R^2 , Adj  R^2 ]$$


In [126]:
#Compute alpha, t-stats, beta, t-stats, R^2, adjusted R^2
def ols1(df):
    y = midprice(df).diff()[1:]/ticksize(df)
    x = VOI(df)[1:]
    x = sm.add_constant(x)
    res = sm.OLS(y,x).fit()
    pred = res.predict(x)
    liss = pd.Series([round(res.params[0],4),round(res.tvalues[0],4),round(res.params[1],4),
            round(res.tvalues[1],4), round(res.rsquared,4), round(res.rsquared_adj,4),np.mean((y - pred)**2)])
    return liss

model1 = df.groupby('Contract').apply(ols1)
model1.columns = ['Alpha', 't(alpha)','beta','t(beta)','R^2','Adj R^2','MSE']
model1

,Alpha,t(alpha),beta,t(beta),R^2,Adj R^2,MSE
Contract,,,,,,,
ag1912,-0.0099,-6.9567,0.0019,30.9035,0.0369,0.0369,0.0477
al1912,-0.0125,-7.7195,0.0020,16.7984,0.0319,0.0318,0.0173
au1912,-0.0031,-1.7184,0.0048,26.2269,0.0384,0.0384,0.0571
bu1912,-0.0020,-2.2557,0.0005,24.9191,0.0279,0.0278,0.0165
cu1912,-0.0019,-1.0909,0.0019,14.1606,0.0159,0.0159,0.0389
fu2001,-0.0033,-3.5001,0.0018,57.8228,0.1199,0.1198,0.0217
hc2001,-0.0081,-4.1727,0.0024,28.6536,0.0358,0.0358,0.0829
ni1912,-0.2300,-30.5948,0.0317,44.5951,0.0717,0.0717,0.7754
pb1912,-0.0058,-1.8697,0.0159,22.3663,0.0566,0.0565,0.0811


### Multifactor Model
We first compute factor Order Imbalance Ratio, Mean Reversion of Midprice and Bid Ask Spread for our Extended Model.

$$[ \alpha, t(\alpha)  , \beta_{OI} , t(\beta_{OI}),\beta_{OIR} , t(\beta_{OIR}),\beta_{MPR} , t(\beta_{MPR}),  R^2 , Adj  R^2 ]$$

In [128]:
#Order Imbalance Ratio
def OIR(df):
    return ((df.BV1 - df.AV1)/(df.BV1 + df.AV1))

#Mid price Reversion
def MPR(df):
    tp = [0]*len(df.index)
    mp = midprice(df)
    tp[0] = midprice(df)[0]
    for t in range(1, len(df.index)):
        if df.volume[t] != df.volume[t-1]:
            tp[t] = (df.turnover[t]-df.turnover[t-1])/((df.volume[t]-df.volume[t-1])*10)
        elif df.volume[t] == df.volume[t-1]:
            tp[t] = tp[t-1]
    return (tp - mp)


def ols2(df):
    y = midprice(df).diff()[1:]/ticksize(df)
    x = pd.DataFrame(VOI(df)/BAS(df))
    x['OIR'] = OIR(df)/BAS(df)
    x['MPR'] = MPR(df)/BAS(df)
    x = sm.add_constant(x)
    res = sm.OLS(y,x[1:]).fit()
    pred = res.predict(x)
    liss = pd.Series([round(res.params.const,4),round(res.tvalues.const,4),
            round(res.params[0],4),round(res.tvalues[0],4), 
            round(res.params.OIR,4),round(res.tvalues.OIR,4), 
            round(res.params.MPR,4),round(res.tvalues.MPR,4),
            round(res.rsquared,4), round(res.rsquared_adj,4),np.mean((y - pred)**2) ])
    return liss

model2 = df.groupby('Contract').apply(ols2)
model2.columns = ['Alpha', 't(alpha)',
                  'beta-OI','t(beta-OI)',
                  'beta-OIR','t(beta-OIR)',
                  'beta-MPR','t(beta-MPR)',
                  'R^2','Adj R^2','MSE']

model2

,Alpha,t(alpha),beta-OI,t(beta-OI),beta-OIR,t(beta-OIR),beta-MPR,t(beta-MPR),R^2,Adj R^2,MSE
Contract,,,,,,,,,,,
ag1912,-0.0408,-1.5089,0.0019,33.2180,-0.1503,-53.4315,0.0000,1.0499,0.1357,0.1356,0.0428
al1912,-0.0154,-0.4323,0.0097,17.1584,-0.3731,-22.8714,-0.0000,-0.1503,0.0875,0.0872,0.0163
au1912,-0.0008,-0.0366,0.0002,27.8895,-0.0076,-45.1958,-0.0000,-0.1395,0.1399,0.1397,0.0511
bu1912,0.0005,0.6428,0.0011,26.5848,-0.1470,-32.8945,-0.0236,-12.9420,0.0911,0.0909,0.0154
cu1912,-0.0086,-0.3739,0.0201,15.8476,-1.1901,-33.5615,-0.0000,-0.2393,0.0977,0.0975,0.0357
fu2001,0.0010,1.1319,0.0017,58.4183,-0.0888,-34.8247,-0.0349,-17.0617,0.1894,0.1893,0.0200
hc2001,-0.0019,-1.0664,0.0026,32.5552,-0.1415,-42.4064,-0.1085,-27.7688,0.1720,0.1719,0.0712
ni1912,0.0210,1.3576,0.3747,46.9342,-5.4959,-48.0525,0.0000,14.5369,0.1381,0.1380,0.7200
pb1912,-0.0289,-1.5308,0.0856,25.0183,-0.9952,-35.3118,-0.0000,-1.3215,0.1796,0.1793,0.0705


Notice that the last model has an average R^2 higher than the Random walk and OI model. This suggest the multifactor model has a better track of the instantaneous Mid price change. 


# Step 5 : Out-of-Sample Modelling
In the last section we only provided result for In-sample modelling. So we are going to compare results of the model using OOS. Here we split the data set into 3 parts. Session AM1, AM2 and PM. Remember for all contracts in SHFE. The trading time is : 
1. AM1 : 09:00:00 - 10:15:00
2. AM2 : 10:30:00 - 11:30:00
3. PM  : 13:30:00 - 15:00:00

The split of the IS:OOS I used is 2:1 Therefore,
1. AM1 : IS **{09:00.00 - 09:50:00}**; OOS **{09:50:00.5 - 10:15:00.0}**
2. AM2 : IS **{10:30.00 - 11:10:00}**; OOS **{11:10:00.5 - 11:30:00.0}**
3. PM  : IS **{13:30.00 - 14:30:00}**; OOS **{14:30:00.5 - 15:30:00.0}**

Finally we take the average of the Out of Sample R^2 and MSE

In [86]:
#define separation time for 3 sessions
sep_time = ['09:50:00.000000','11:10:00.000000','14:30.00.000000']
open_time = ['09:00:000.000000','10:30:00.000000','13:30.00.000000']
close_time = ['10:15:00.000000','11:30:00.000000','15:00:00.000000']
def ols_mse_oos(df):
    #Set variables
    MSE =[]
    Rsquared = []
    for i in range(0,3):
    #Separate In sample & Out of sample    
        iss = df[(df.index > open_time[i]) & (df.index < sep_time[i])]
        oos = df[(df.index > sep_time[i]) & (df.index < close_time[i])]
    #Compute results
        y = midprice(iss).diff()[1:]/ticksize(iss)
        new_y = midprice(oos).diff()[1:]/ticksize(oos)
        x = np.array([1]*(len(iss)-1)) 
        newx = np.array([1]*(len(oos)-1))
        res = sm.OLS(y,x).fit()
        pred = res.predict(newx)
        MSE.append(np.mean((new_y-pred)**2))
        Rsquared.append(res.rsquared)
    
    liss = pd.Series([np.mean(MSE),np.mean(Rsquared)])
    return liss

oos = df.groupby('Contract').apply(ols_mse_oos)
oos.columns = ['MSE','R^2']
print(oos)

            MSE     R^2
Contract               
ag1912   0.0372 -0.0000
al1912   0.0131 -0.0000
au1912   0.0363  0.0000
bu1912   0.0142  0.0000
cu1912   0.0281  0.0000
fu2001   0.0238 -0.0000
hc2001   0.0826  0.0000
ni1912   0.9827 -0.0000
pb1912   0.0703 -0.0000
rb2001   0.0411 -0.0000
ru2001   0.0306  0.0000
sn2001   0.8567  0.0000
sp2001   0.0187  0.0000
ss2002   0.1754 -0.0000
wr2001   3.7120 -0.0000
zn1912   0.0412 -0.0000


In [100]:
def ols_mse1_oos(df):
    MSE =[]
    Rsquared = []
    for i in [0,1,2]:
    #Separate In sample & Out of sample    
        iss = df[(df.index > open_time[i]) & (df.index < sep_time[i])]
        oos = df[(df.index > sep_time[i]) & (df.index < close_time[i])]
    #Compute results
        y = midprice(iss).diff()[1:]/ticksize(iss)
        new_y = midprice(oos).diff()[1:]/ticksize(oos)
        x = VOI(iss)[1:]
        x = sm.add_constant(x) 
        newx = VOI(oos)[1:]
        newx = sm.add_constant(newx)
        res = sm.OLS(y,x).fit()
        pred = res.predict(newx)
        MSE.append(np.mean((new_y - pred)**2))
        Rsquared.append(res.rsquared)
    liss = pd.Series([np.mean(MSE),np.mean(Rsquared)])
    return liss

oos1 = df.groupby('Contract').apply(ols_mse1_oos)
oos1.columns = ['MSE','R^2']
print(oos1)

            MSE    R^2
Contract              
ag1912   0.0370 0.0365
al1912   0.0160 0.0415
au1912   0.0361 0.0480
bu1912   0.0147 0.0543
cu1912   0.0281 0.0180
fu2001   0.0313 0.1289
hc2001   0.0845 0.0391
ni1912   3.7103 0.0789
pb1912   0.0680 0.0652
rb2001   0.0407 0.0342
ru2001   0.0291 0.0506
sn2001   0.9796 0.0750
sp2001   0.0177 0.0570
ss2002   0.1716 0.0423
wr2001   4.0818 0.0241
zn1912   0.0402 0.0428


In [123]:
def ols_mse2_oos(df):
    MSE =[]
    Rsquared = []
    for i in [0,1,2]:
    #Separate In sample & Out of sample    
        iss = df[(df.index > open_time[i]) & (df.index < sep_time[i])]
        oos = df[(df.index > sep_time[i]) & (df.index < close_time[i])]
    #Compute results
        y = midprice(iss).diff()[1:]/ticksize(iss)
        new_y = midprice(oos).diff()[1:]/ticksize(oos)
        x = pd.DataFrame(VOI(iss)/BAS(iss))
        x['OIR'] = OIR(iss)/BAS(iss)
        x['MPR'] = MPR(iss)/BAS(iss)
        x = sm.add_constant(x) 
        newx = pd.DataFrame(VOI(oos)/BAS(oos))
        newx['OIR'] = OIR(oos)/BAS(oos)
        newx['MPR'] = MPR(oos)/BAS(oos)
        newx = sm.add_constant(newx)
        res = sm.OLS(y,x[1:]).fit()
        pred = res.predict(newx[1:])
        MSE.append(np.mean((new_y - pred)**2))
        Rsquared.append(res.rsquared)
    liss = pd.Series([np.mean(MSE),np.mean(Rsquared)])
    return liss

oos2 = df.groupby('Contract').apply(ols_mse2_oos)
oos2.columns = ['MSE','R^2']
print(oos2)

            MSE    R^2
Contract              
ag1912   0.0361 0.1475
al1912   0.0170 0.1155
au1912   0.0335 0.1625
bu1912   0.0138 0.1187
cu1912   0.0258 0.1013
fu2001   0.0304 0.1934
hc2001   0.0728 0.1687
ni1912   3.7102 0.1532
pb1912   0.0608 0.1895
rb2001   0.0363 0.1705
ru2001   0.0259 0.1882
sn2001   0.8612 0.1068
sp2001   0.0166 0.1085
ss2002   0.1601 0.1183
wr2001   4.6998 0.1245
zn1912   0.0359 0.1697
